In [151]:
# !pip install tensorflow

In [152]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
import tensorflow as tf

In [153]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [154]:
train

,ID,Exercise_Duration,Body_Temperature(F),BPM,Height(Feet),Height(Remainder_Inches),Weight(lb),Weight_Status,Gender,Age,Calories_Burned
0,TRAIN_0000,26.0,105.6,107.0,5.0,9.0,154.3,Normal Weight,F,45,166.0
1,TRAIN_0001,7.0,103.3,88.0,6.0,6.0,224.9,Overweight,M,50,33.0
2,TRAIN_0002,7.0,103.3,86.0,6.0,3.0,218.3,Overweight,M,29,23.0
3,TRAIN_0003,17.0,104.0,99.0,5.0,6.0,147.7,Normal Weight,F,33,91.0
4,TRAIN_0004,9.0,102.7,88.0,5.0,10.0,169.8,Normal Weight,M,38,32.0
...,...,...,...,...,...,...,...,...,...,...,...
7495,TRAIN_7495,22.0,105.1,104.0,4.0,10.0,112.4,Normal Weight,F,75,151.0
7496,TRAIN_7496,20.0,105.3,104.0,5.0,8.0,147.7,Normal Weight,F,21,114.0
7497,TRAIN_7497,8.0,103.1,90.0,6.0,2.0,202.8,Overweight,M,57,41.0
7498,TRAIN_7498,12.0,104.4,97.0,5.0,9.0,167.6,Overweight,M,35,57.0


In [155]:
train['Height(Cm)'] = (train['Height(Feet)']*30.48) + (train['Height(Remainder_Inches)']*2.54)
train['Weight(Kg)'] = round(train['Weight(lb)']*0.453592 , 2)
# train['BMI'] = round(train['Weight(Kg)'] / (round(train['Height(Cm)'] *0.01,2)**2),2)

In [156]:
성별 = [train["Gender"].str.contains("M")]
choicelist1 = ['1']
train['Gender'] = np.select(성별, choicelist1, default='0')
train['Gender'] = train['Gender'].astype(int)

In [157]:
test['Height(Cm)'] = (test['Height(Feet)']*30.48) + (test['Height(Remainder_Inches)']*2.54)
test['Weight(Kg)'] = round(test['Weight(lb)']*0.453592 , 2)
# test['BMI'] = round(test['Weight(Kg)'] / (round(test['Height(Cm)'] *0.01,2)**2),2)

In [158]:
성별 = [test["Gender"].str.contains("M")]
choicelist1 = ['1']
test['Gender'] = np.select(성별, choicelist1, default='0')
test['Gender'] = test['Gender'].astype(int)

In [159]:
train = train[['Exercise_Duration', 'Body_Temperature(F)', 'BPM', 'Gender', 'Age','Height(Cm)', 'Weight(Kg)', 'Calories_Burned']]
train

,Exercise_Duration,Body_Temperature(F),BPM,Gender,Age,Height(Cm),Weight(Kg),Calories_Burned
0,26.0,105.6,107.0,0,45,175.26,69.99,166.0
1,7.0,103.3,88.0,1,50,198.12,102.01,33.0
2,7.0,103.3,86.0,1,29,190.50,99.02,23.0
3,17.0,104.0,99.0,0,33,167.64,67.00,91.0
4,9.0,102.7,88.0,1,38,177.80,77.02,32.0
...,...,...,...,...,...,...,...,...
7495,22.0,105.1,104.0,0,75,147.32,50.98,151.0
7496,20.0,105.3,104.0,0,21,172.72,67.00,114.0
7497,8.0,103.1,90.0,1,57,187.96,91.99,41.0
7498,12.0,104.4,97.0,1,35,175.26,76.02,57.0


In [160]:
test = test[['Exercise_Duration', 'Body_Temperature(F)', 'BPM', 'Gender', 'Age','Height(Cm)', 'Weight(Kg)']]
test

,Exercise_Duration,Body_Temperature(F),BPM,Gender,Age,Height(Cm),Weight(Kg)
0,26.0,105.1,107.0,0,45,154.94,51.98
1,29.0,104.9,111.0,1,21,187.96,89.99
2,11.0,104.0,90.0,1,58,175.26,77.02
3,24.0,105.4,108.0,1,35,182.88,88.99
4,29.0,106.0,116.0,0,67,170.18,67.00
...,...,...,...,...,...,...,...
7495,28.0,105.6,106.0,1,51,182.88,81.01
7496,4.0,102.0,84.0,1,25,177.80,83.01
7497,24.0,105.1,97.0,0,42,157.48,59.01
7498,8.0,103.1,83.0,0,48,165.10,61.01


# 딥러닝

In [161]:
label = "Calories_Burned"
label

'Calories_Burned'

In [162]:
X_train = train.drop(columns=label).sample(frac=0.8, random_state=24)
X_test = train.drop(columns=label).drop(index=X_train.index)
y_train = train[label].loc[X_train.index]
y_test = train[label].loc[X_test.index]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6000, 7), (1500, 7), (6000,), (1500,))

In [163]:
input_shape = (X_train.shape[1],)
input_shape

(7,)

In [164]:
model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=X_train.iloc[0].shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)])

In [165]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [166]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_8 (Batc  (None, 7)                28        
 hNormalization)                                                 
                                                                 
 dense_31 (Dense)            (None, 64)                512       
                                                                 
 dense_32 (Dense)            (None, 32)                2080      
                                                                 
 dense_33 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,653
Trainable params: 2,639
Non-trainable params: 14
_________________________________________________________________


In [167]:
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

In [168]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=150)

In [169]:
history = model.fit(X_train, y_train, 
                    epochs=500, verbose=0, 
                    validation_split=0.1,
                    callbacks=[early_stop, PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................

In [170]:
df_hist = pd.DataFrame(history.history)
df_hist

,loss,val_loss
0,8160.368652,225150.343750
1,370.821014,7441.984863
2,271.509918,209.393158
3,271.832062,89.239357
4,244.152710,78.078674
...,...,...
495,131.595123,11.252931
496,154.144272,9.902607
497,162.604736,15.204217
498,143.498138,16.092117


In [171]:
# df_hist.to_csv('df_hist.csv')

In [172]:
# 실행시 커널 죽을수도 있음..!
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
# df_hist[["loss", "val_loss"]].plot(ax=axes[0],ylim=(0,200))
# df_hist[["mae", "val_mae"]].plot(ax=axes[1],ylim=(0,10))

In [173]:
mse = model.evaluate(X_test, y_test)
mse

47/47 [==============================] - 0s 826us/step - loss: 23.9527


23.95271873474121

In [174]:
rmse = mse ** 0.5
rmse

4.894151482610772

In [175]:
predictions = model.predict(test)
predictions

235/235 [==============================] - 0s 585us/step


array([[178.0719  ],
       [194.4879  ],
       [ 55.67518 ],
       ...,
       [135.84685 ],
       [ 31.775782],
       [188.71622 ]], dtype=float32)

In [176]:
submission['Calories_Burned'] = predictions
submission['Calories_Burned']

0       178.071899
1       194.487900
2        55.675179
3       168.729340
4       225.860687
           ...    
7495    197.994232
7496     12.350112
7497    135.846848
7498     31.775782
7499    188.716217
Name: Calories_Burned, Length: 7500, dtype: float32

In [177]:
submission.to_csv(f'dl_tf_{rmse}.csv', index = False)